In [3]:
import requests
from bs4 import BeautifulSoup as bs, Tag
import re
import pandas as pd
import time

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_exports'
page = requests.get(url)
response = requests.get(url)
html = response.text
status = response.status_code
if status == 200:
    page = response.text
    soup = bs(page)
else:
    print(f"Oops! Received status code {status}")

In [13]:
 table = soup.find_all('table')[0]


In [15]:
column_names = table.find_all('th')
column_names

[<th>Country
 </th>,
 <th>Exports
 </th>,
 <th>Year
 </th>,
 <th>Top goods export (2021)<sup class="reference" id="cite_ref-2"><a href="#cite_note-2"><span class="cite-bracket">[</span>2<span class="cite-bracket">]</span></a></sup><sup class="noprint Inline-Template" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Manual_of_Style/Dates_and_numbers#Chronological_items" title="Wikipedia:Manual of Style/Dates and numbers"><span title="2022 data is available; see source given (January 2025)">needs update</span></a></i>]</sup>
 </th>]

In [19]:
column_names = [title.text.strip() for title in column_names]
column_names

['Country', 'Exports', 'Year', 'Top goods export (2021)[2][needs update]']

In [49]:
def clean_text(text):
    # Remove all [bracketed text]
    return re.sub(r'\[.*?\]', '', text).strip()

cleaned_headers = [clean_text(h) for h in column_names]


['Country', 'Exports', 'Year', 'Top goods export (2021)']


In [59]:
df = pd.DataFrame(columns = cleaned_headers)
df


,Country,Exports,Year,Top goods export (2021)


In [29]:
data_row = table.find_all('tr')

In [67]:
 def clean_citations(text):
    return re.sub(r'\[\d+\]', '', text).strip()

cleaned_data = []

for row in data_row[1:]:
    if isinstance(row, str):
        parsed = bs(row, 'html.parser')
    elif isinstance(row, Tag):
        parsed = row
    else:
        continue

    row_data = parsed.find_all('td')
    table_rows = [clean_citations(data.text) for data in row_data]
    cleaned_data.append(table_rows)

print(cleaned_data)


[['China', '3,511,248', '2023', 'Broadcasting equipment'], ['United States', '3,051,824', '2023', 'Petroleum'], ['Germany', '2,104,251', '2023', 'Cars'], ['United Kingdom', '1,074,781', '2023', 'Gold'], ['France', '1,051,679', '2023', 'Packaged medications'], ['Netherlands', '949,983', '2023', 'Petroleum'], ['Japan', '920,737', '2023', 'Cars'], ['Italy', '793,588', '2023', 'Packaged medications'], ['Singapore', '778,000', '2023', 'Integrated circuits'], ['India', '773,223', '2023', 'Petroleum'], ['South Korea', '769,534', '2023', 'Integrated circuits'], ['United Arab Emirates', '753,000', '2022', 'Petroleum'], ['Ireland', '731,813', '2023', 'Serums and vaccines'], ['Canada', '717,677', '2023', 'Petroleum'], ['Hong Kong (CN)', '673,305', '2023', 'Integrated circuits'], ['Switzerland', '661,627', '2023', 'Gold'], ['Mexico', '649,312', '2023', 'Cars'], ['Spain', '615,829', '2023', 'Cars'], ['Taiwan', '536,128', '2022', 'Integrated circuits'], ['Belgium', '535,173', '2023', 'Vaccines'], ['

In [69]:
for row in cleaned_data:
    df.loc[len(df)] = row

In [77]:
df

,Country,Exports,Year,Top goods export (2021)
0,China,"3,511,248",2023,Broadcasting equipment
1,United States,"3,051,824",2023,Petroleum
2,Germany,"2,104,251",2023,Cars
3,United Kingdom,"1,074,781",2023,Gold
4,France,"1,051,679",2023,Packaged medications
...,...,...,...,...
405,Tonga,60,2022,Shellfish
406,Nauru,31,2018,Fish
407,Palau,12,2021,Computers
408,Kiribati,11,2021,Fish


In [81]:
df.to_csv(r'C:\Users\pmush\OneDrive\Desktop\Data Science Prog\webscrapping\exports.csv', index = False)